### Project Problem Statement

**Business Problem:** Our subscription service is losing customers (churning), which costs the company money.

**Goal:** Build a system that automatically identifies the users **most likely to churn** in the near future.

**Action:** This system will provide a prioritized list to the retention team, allowing them to proactively send offers (like discounts) to save these at-risk customers, optimizing their budget and reducing overall churn.

### **Phase 1: Setup & Data Generation (The Foundation)**

### **Step 1: Imports and Setup**

First, we'll import the necessary libraries and set up our parameters. We'll also create the `data/` directory to store our CSVs.

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta 
import os

# Parameters
NUM_USERS = 10000
NUM_EVENTS = 500000
NUM_TICKETS = 4000
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 10, 18) # Today
CHURN_RATE = 0.05 # 5% churn rate

if not os.path.exists('data'):
    os.makedirs('data')

print("Libraries imported and parameters set.")

Libraries imported and parameters set.


### **Step 2: Generate `user_subscriptions.csv`**

This creates our main list of users, their subscription plans, and when they signed up. We will also pre-determine who our "churners" will be so we can make their activity data look different.

In [8]:
# Create a list of user IDs
user_ids = range(1, NUM_USERS+1)

# Assign churn status
churn_status = np.random.choice([True, False], size=NUM_USERS, p= [CHURN_RATE,1-CHURN_RATE])

# Generate User Data

users_data = []
for user_id in user_ids:
    plan = np.random.choice(["Basic","Premium","Family"])
    billing_cycle = "Annual" if plan == "Family" else random.choice(["Monthly","Annual"])

    # signups are spread out over the last year
    signup_date = START_DATE + timedelta(days = random.randint(0,365))

    users_data.append({
        "user_id" : user_id,
        "plan_type" : plan,
        "billing_cycle" : billing_cycle,
        "signup_date" : signup_date,
        "is_churner" : churn_status[user_id - 1]
    })
                                         

# Create DataFrame and save
df_users = pd.DataFrame(users_data)
df_users.to_csv("data/user_subscriptions.csv", index=False)

print(f"Generated data/user_subscriptions.csv with {len(df_users)} users.")
print(df_users.head()) 

Generated data/user_subscriptions.csv with 10000 users.
   user_id plan_type billing_cycle signup_date  is_churner
0        1     Basic        Annual  2024-08-25       False
1        2     Basic       Monthly  2024-12-23       False
2        3     Basic       Monthly  2024-11-11       False
3        4     Basic        Annual  2024-11-22       False
4        5     Basic        Annual  2024-08-01       False


In [16]:
df_users.shape

(10000, 5)

### **Step 3: Generate `user_activity_logs.csv`**

This is the most important part. We will generate a large log file.

  * **Active users** (`is_churner=False`) will have consistent activity right up to `END_DATE`.
  * **Churned users** (`is_churner=True`) will have their activity stop at least 30 days before `END_DATE`.

<!-- end list -->


In [9]:
churn_user_ids = df_users[df_users['is_churner'] == True]['user_id'].tolist()
active_user_ids = df_users[df_users['is_churner'] == False]['user_id'].tolist()

event_types = ['login', 'played_video', 'search', 'downloaded_offline', 'profile_update']

# --- Generate Event Data ---
events_data = []

# Generate for active users
active_events_count = int(NUM_EVENTS * 0.9) # Active users are more active
for _ in range(active_events_count):
    user_id = random.choice(active_user_ids)
    # Active users have events up to today
    event_date = START_DATE + timedelta(days=random.randint(0, (END_DATE - START_DATE).days))
    events_data.append({
        'user_id': user_id,
        'timestamp': event_date,
        'event_type': random.choice(event_types)
    })

# Generate for churn users
churn_events_count = int(NUM_EVENTS * 0.1) # Churn users are less active
for _ in range(churn_events_count):
    user_id = random.choice(churn_user_ids)
    # Churn users' activity stops at least 30 days ago
    churn_end_day = (END_DATE - START_DATE).days - 30
    event_date = START_DATE + timedelta(days=random.randint(0, churn_end_day))
    events_data.append({
        'user_id': user_id,
        'timestamp': event_date,
        'event_type': random.choice(event_types)
    })

# Create DataFrame, sort, and save
df_events = pd.DataFrame(events_data)
df_events = df_events.sort_values(by='timestamp')
df_events.to_csv('data/user_activity_logs.csv', index=False)

print(f"Generated data/user_activity_logs.csv with {len(df_events)} events.")
print(df_events.head())

Generated data/user_activity_logs.csv with 500000 events.
        user_id  timestamp      event_type
155673     7791 2024-01-01          search
208293     3097 2024-01-01           login
40935      7882 2024-01-01           login
93732      7402 2024-01-01  profile_update
426832     6441 2024-01-01  profile_update


In [10]:
df_events.shape

(500000, 3)

In [13]:
df_events["event_type"].value_counts()

event_type
played_video          100268
downloaded_offline    100190
login                 100075
search                 99975
profile_update         99492
Name: count, dtype: int64

### **Step 4: Generate `customer_support.csv`**

Finally, let's create support tickets. We'll make our churners more likely to have "Billing" issues, which is a realistic pattern.

In [14]:
ticket_types = ['Technical', 'Billing', 'Account Access', 'General Inquiry']

# Generate Ticket Data 
tickets_data = []
for i in range(NUM_TICKETS):
    # Make churners more likely to submit tickets
    if random.random() < 0.3: # 30% of tickets come from the 5% of users who churn
        user_id = random.choice(churn_user_ids)
        issue = random.choices(ticket_types, weights=[0.3, 0.5, 0.1, 0.1], k=1)[0] # Higher chance of billing
    else: # 70% of tickets from active users
        user_id = random.choice(active_user_ids)
        issue = random.choices(ticket_types, weights=[0.4, 0.1, 0.2, 0.3], k=1)[0] # Lower chance of billing

    ticket_date = START_DATE + timedelta(days=random.randint(0, (END_DATE - START_DATE).days))
    
    tickets_data.append({
        'ticket_id': 1000 + i,
        'user_id': user_id,
        'date_opened': ticket_date.date(),
        'issue_type': issue
    })

# Create DataFrame and save
df_tickets = pd.DataFrame(tickets_data)
df_tickets.to_csv('data/customer_support.csv', index=False)

print(f"Generated data/customer_support.csv with {len(df_tickets)} tickets.")
print(df_tickets.head())

Generated data/customer_support.csv with 4000 tickets.
   ticket_id  user_id date_opened       issue_type
0       1000     8388  2025-08-25   Account Access
1       1001     6393  2024-01-19  General Inquiry
2       1002     8596  2025-09-19   Account Access
3       1003      594  2025-07-24          Billing
4       1004     2140  2024-08-31          Billing


In [15]:
df_tickets.shape

(4000, 4)

### Phase 1 Summary

In this phase, **we** successfully built the entire foundation for our project.
1.  **Environment Setup:** We created a professional project structure with a dedicated virtual environment (`venv`) to manage our packages.
2.  **Data Simulation:** Instead of using a simple, pre-made file, we wrote a Python script to **simulate a realistic, multi-source company database**.
3.  **Data Creation:** We generated three distinct, interconnected datasets (`user_subscriptions.csv`, `user_activity_logs.csv`, and `customer_support.csv`) that mimic how real user data is stored.
4.  **Realistic Patterns:** Most importantly, we embedded **realistic business logic** into the data (e.g., churners stop being active and are more likely to have billing issues).

***
